# Tau Notebook

In [ ]:
import os

RunningInCOLAB = 'google.colab' in str(get_ipython())  # checks to see if we are in google colab
if RunningInCOLAB:                                     # installs packages and repo if in colab
    !pip install ipywidgets &> /dev/null               
    !pip install neuron &> /dev/null

    !git clone https://github.com/GregGlickert/Neural-Modeling-Manual.git &> /dev/null 
    %cd Neural-Modeling-Manual/Section3-Synaptic-Design/Section3a-Time-Constants/

if(os.path.exists('x86_64')==False):        
    os.system('nrnivmodl modfiles') # compile modfiles. Return 0 for success, 1 for failure.

In [ ]:
from neuron import h
from neuron.units import ms, mV
import matplotlib.pyplot as plt
from matplotlib import gridspec
import ipywidgets as widgets
from ipywidgets import interactive_output,HBox,VBox,Label,Layout
from IPython.display import display
from IPython.display import clear_output


def model(weight,input_freqency,tau_r_AMPA,tau_d_AMPA,tau_r_NMDA,tau_d_NMDA):
    h.load_file('stdrun.hoc')
    h.load_file('PN_cells.hoc') # reads in template
    cell = h.PN_C() #cell from template file

    h.dt = 0.1 # time step (resolution) of the simulation in ms
    h.tstop = 2000 # how long to run the simulation in ms
    h.v_init= -70 # initial membrane potential in mV
    
    t = h.Vector()      # Vectors are used to record data in the simulation
    soma_v = h.Vector()
    i = h.Vector()

    conn = h.AMPA_NMDA(cell.dend[0](0.9)) # makes syn connection
   

    conn.tau_r_AMPA = tau_r_AMPA # tau raise
    conn.tau_d_AMPA = tau_d_AMPA # tau decay
    conn.tau_r_NMDA = tau_r_NMDA
    conn.tau_d_NMDA = tau_d_NMDA
    conn.initW = weight

    nstim = h.NetStim(0.9)               # creating a stimulation for the cell

    nstim.interval=1000/input_freqency   # Hz of input
    nstim.start=500                      # when stim should start
    nstim.number = 100                   # 
    nstim.noise = 0                      # level of noise
    nc = h.NetCon(nstim,conn,10,10,1) # makes connections with netcon weight as 1 real weight is inside synapse
    
    
    
    t.record(h._ref_t) # derefences and records data
    soma_v.record(cell.soma[0](0.5)._ref_v)
    i.record(conn._ref_i)

    h.finitialize(h.v_init * mV)  # runs sims
    h.continuerun(h.tstop * ms)   
    
    plot_model(t,soma_v,i)
    
def plot_model(t,soma_v,i):         # ploting code
    fig = plt.figure(figsize=(8, 8),tight_layout=True) 
    gs = gridspec.GridSpec(2, 1, width_ratios=[1],height_ratios=[1,1]) 
    
    ax0 = plt.subplot(gs[0,0])  # top plot
    ax1 = plt.subplot(gs[1,0])  # bottom plot
    
    ax0.plot(t ,soma_v)
    ax0.set_title('soma voltage')   
    ax0.set_xlim(200,2200)
    
    ax1.plot(t,i)
    ax1.set_title('synaptic current')
    ax1.set_xlim(200,2200)
    
    plt.show()
    
weight = 1         # inital values used by the slider
input_freqency = 7  
tau_r_AMPA = 0.43
tau_d_AMPA = 10.8
tau_r_NMDA = 3.7
tau_d_NMDA = 125

# widgets the get displayed below
w_run = widgets.Button(description='Run',icon='history',button_style='primary')    
w_weight = widgets.FloatSlider(value=weight,min=0.001,max=10, step=0.2,description='weight')
w_input_freqency = widgets.FloatSlider(value=input_freqency,min=0.001,max=50, step=1,description='Input Freq')
w_tau_r_AMPA = widgets.FloatSlider(value=tau_r_AMPA,min=0,max=10, step=0.01,description='tau rise AMPA')
w_tau_d_AMPA = widgets.FloatSlider(value=tau_d_AMPA,min=0,max=200, step=10,description='tau decay AMPA')
w_tau_r_NMDA = widgets.FloatSlider(value=tau_r_NMDA,min=0,max=10, step=0.01,description='tau rise AMPA')
w_tau_d_NMDA = widgets.FloatSlider(value=tau_d_NMDA,min=0,max=200, step=10,description='tau decay AMPA')


def run_model(*arges):
    clear_output()  #               Removes the sliders and then updated the values and displays the new valuesgit
    
    weight = w_weight.value
    input_freqency = w_input_freqency.value
    tau_r_AMPA = w_tau_r_AMPA.value
    tau_d_AMPA = w_tau_d_AMPA.value
    tau_r_NMDA = w_tau_r_NMDA.value
    tau_d_NMDA = w_tau_d_NMDA.value
    display(ui)
    model(weight,input_freqency,tau_r_AMPA,tau_d_AMPA,tau_r_NMDA,tau_d_NMDA)



w_run.on_click(run_model)

ui = VBox([HBox([w_run]), HBox([w_weight,w_tau_r_AMPA,w_tau_r_NMDA]),HBox([w_input_freqency,w_tau_d_AMPA, w_tau_d_NMDA])])
ui